# Lab | Text Generation from Shakespeare's Sonnet

This notebook explores the fascinating domain of text generation using a deep learning model trained on Shakespeare's sonnets. 

The objective is to create a neural network capable of generating text sequences that mimic the style and language of Shakespeare.

By utilizing a Recurrent Neural Network (RNN) with Long Short-Term Memory (LSTM) layers, this project aims to demonstrate how a model can learn and replicate the complex patterns of early modern English. 

The dataset used consists of Shakespeare's sonnets, which are preprocessed and tokenized to serve as input for the model.

Throughout this notebook, you will see the steps taken to prepare the data, build and train the model, and evaluate its performance in generating text. 

This lab provides a hands-on approach to understanding the intricacies of natural language processing (NLP) and the potential of machine learning in creative text generation.

Let's import necessary libraries

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

!pip install --upgrade tensorflow

Let's get the data!

In [5]:
import requests
url = 'https://raw.githubusercontent.com/martin-gorner/tensorflow-rnn-shakespeare/master/shakespeare/sonnets.txt'
resp = requests.get(url)
with open('sonnets.txt', 'wb') as f:
    f.write(resp.content)

data = open('sonnets.txt').read()

corpus = data.lower().split("\n")

In [6]:
corpus[1:50]

['',
 '',
 '',
 'to the only begetter of',
 'these insuing sonnets',
 'mr. w. h. all happiness',
 'and that eternity',
 'promised by',
 'our ever-living poet wisheth',
 'the well-wishing',
 'adventurer in',
 'setting forth',
 't. t.',
 '',
 '',
 'i.',
 '',
 'from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 '  pity the world, or else this glutton be,',
 "  to eat the world's due, by the grave and thee.",
 '',
 'ii.',
 '',
 'when forty winters shall beseige thy brow,',
 "and d

Step 1: Initialise a tokenizer and fit it on the corpus variable using .fit_on_texts

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(corpus)


Step 2: Calculate the Vocabulary Size

Let's figure out how many unique words are in your corpus. This will be the size of your vocabulary.

Calculate the length of tokenizer.word_index, add 1 to it and store it in a variable called total_words.

In [8]:
total_words = len(tokenizer.word_index) + 1  # Add 1 to include the padding token

print(total_words)

3375


Create an empty list called input_sequences.

For each sentence in your corpus, convert the text into a sequence of integers using the tokenizer.
Then, generate n-gram sequences from these tokens.

Store the result in the list input_sequences.

In [9]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] 
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]  # Generate n-gram sequence
        input_sequences.append(n_gram_sequence)

input_sequences[0:20]        


[[3, 2],
 [3, 2, 313],
 [3, 2, 313, 1375],
 [3, 2, 313, 1375, 4],
 [118, 1376],
 [118, 1376, 878],
 [1377, 1378],
 [1377, 1378, 1379],
 [1377, 1378, 1379, 23],
 [1377, 1378, 1379, 23, 1380],
 [1, 8],
 [1, 8, 517],
 [1381, 30],
 [126, 186],
 [126, 186, 278],
 [126, 186, 278, 635],
 [126, 186, 278, 635, 1382],
 [2, 98],
 [2, 98, 879],
 [1383, 7]]

Calculate the length of the longest sequence in input_sequences. Assign the result to a variable called max_sequence_len.

Now pad the sequences using pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre').
Convert it to a numpy array and assign the result back to our variable called input_sequences.

In [10]:
max_sequence_len = max([len(seq) for seq in input_sequences])

from tensorflow.keras.preprocessing.sequence import pad_sequences

input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [11]:
input_sequences.shape()

TypeError: 'tuple' object is not callable

Prepare Predictors and Labels

Split the sequences into two parts:

- Predictors: All elements from input_sequences except the last one.
- Labels: The last element of each sequence in input_sequences.

In [10]:
predictors = input_sequences[:, :-1]  # Select all columns except the last one
labels = input_sequences[:, -1]  # Select the last column
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=total_words)
print(input_sequences.shape)
labels[10:40]

(15484, 11)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

: 

One-Hot Encode the Labels :

Convert the labels (which are integers) into one-hot encoded vectors. 

Ensure the length of these vectors matches the total number of unique words in your vocabulary.

Use ku.to_categorical() on labels with num_classes = total_words

Assign the result back to our variable labels.

In [ ]:
# Debugging
print("Total words in vocabulary:", total_words)
print("Labels shape:", labels.shape)
print("Labels data type:", labels.dtype)
labels = to_categorical(labels, num_classes=total_words)
print("One-hot encoding complete.")

# from tensorflow.keras.utils import to_categorical
# labels = to_categorical(labels, num_classes=total_words)

import numpy as np
from tensorflow.keras.utils import to_categorical

def batch_to_categorical(labels, num_classes, batch_size=1000):
    one_hot_batches = []  # Liste zur Speicherung der Batches
    for i in range(0, len(labels), batch_size):
        batch_labels = labels[i:i+batch_size]  # Extrahiere die aktuelle Batch
        one_hot_batch = to_categorical(batch_labels, num_classes=num_classes)  # One-Hot-Encoding
        one_hot_batches.append(one_hot_batch)  # Speichere das Ergebnis
    
    # Füge alle Batches in ein großes Array zusammen
    return np.vstack(one_hot_batches)

# Beispielaufruf:
batch_size = 1000
labels_one_hot = batch_to_categorical(labels, num_classes=total_words, batch_size=batch_size)


Total words in vocabulary: 3375
Labels shape: (15484, 3375)
Labels data type: float64


# Initialize the Model

Start by creating a Sequential model.

Add Layers to the Model:

Embedding Layer: The first layer is an embedding layer. It converts word indices into dense vectors of fixed size (100 in this case). Set the input length to the maximum sequence length minus one, which corresponds to the number of previous words the model will consider when predicting the next word.

Bidirectional LSTM Layer: Add a Bidirectional LSTM layer with 150 units. This layer allows the model to learn context from both directions (past and future) in the sequence. return_sequences=True

Dropout Layer: Add a dropout layer with a rate of 0.2 to prevent overfitting by randomly setting 20% of the input units to 0 during training.

LSTM Layer: Add a second LSTM layer with 100 units. This layer processes the sequence and passes its output to the next layer.

Dense Layer (Intermediate): Add a dense layer with half the total number of words as units, using ReLU activation. A regularization term (L2) is added to prevent overfitting.

Dense Layer (Output): The final dense layer has as many units as there are words in the vocabulary, with a softmax activation function to output a probability distribution over all words.

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.regularizers import l2

model = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len - 1),
    Bidirectional(LSTM(150, return_sequences=True)),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words // 2, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(total_words, activation='softmax')
])

# Compile the Model:

Compile the model using categorical crossentropy as the loss function, the Adam optimizer for efficient training, and accuracy as the metric to evaluate during training.

In [10]:
optimizer='adam'
loss='categorical_crossentropy'
metrics=['accuracy']

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



## Training the Model'

In [ ]:
history = model.fit(predictors, labels, epochs=50, batch_size=64, verbose=1)

# Print Model Summary:

Use model.summary() to print a summary of the model, which shows the layers, their output shapes, and the number of parameters.

In [12]:
model.summary()

NameError: name 'model' is not defined

# Now train the model for 50 epochs and assign it to a variable called history.

Training the model with 50 epochs should get you around 40% accuracy.

You can train the model for as many epochs as you like depending on the time and computing constraints you are facing. Ideally train it for a larger amount of epochs than 50.

That way you will get better text generation at the end.

However, dont waste your time.

In [12]:
# Your code here :

# Use plt from matplotlib to plot the training accuracy over epochs and the loss over epochs

First you will have to get the accuracy and loss data over epochs, you can do this by using methods on your model.

In [13]:
# Your code here :

# Generate text with the model based on a seed text

Now you will create two variables :

- seed_text = 'Write the text you want the model to use as a starting point to generate the next words'
- next_words = number_of_words_you_want_the_model_to_generate

Please change number_of_words_you_want_the_model_to_generate by an actual integer.

In [14]:
# Your code here :

Now create a loop that runs based on the next_words variable and generates new text based on your seed_text input string. Print the full text with the generated text at the end.

This time you dont get detailed instructions.

Have fun!

In [15]:
# Your code here :

Experiment with at least 3 different seed_text strings and see what happens!

In [16]:
# Your code here :